# Timeout & retry functions in Python

#### The following are implementations on how to specify timeouts on functions that run for too long.  Additionally by using the ```retrying``` library, we are able to retry the function a specific number of times.

### 1. signal.SIGALRM

Define a ```Timeout``` class that will raise an exception after the containing code does not finish within a specified number of seconds.  Use the ```retry``` decorator to retry the function a specified number of times after the timeout.

<div class="alert alert-danger">
  <strong>Note!</strong> You cannot use SIGALRM within multi-threaded applications (only in the main thread) and SIGARLM is only supported on Unix platforms.  This is NOT a recommended approach.
</div>

In [1]:
import signal


class TimeoutException(Exception):
    pass

class Timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
        
    def handle_timeout(self, signum, frame):
        raise TimeoutException(self.error_message)
        
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)
        
    def __exit__(self, type, value, traceback):
        signal.alarm(0)

In [2]:
import time
from random import randint
from retrying import retry


@retry(stop_max_attempt_number=6)  # stop after 6 attempts
def long_running_function():
    with Timeout(seconds=1):
        sleep_time = randint(0,9)
        print("Doing something that takes " + str(sleep_time) + " seconds...")
        time.sleep(sleep_time)
        print("Finished!")


try:
    long_running_function()
except TimeoutException:
    print("Timeout!")

Doing something that takes 5 seconds...
Doing something that takes 1 seconds...
Doing something that takes 9 seconds...
Doing something that takes 3 seconds...
Doing something that takes 2 seconds...
Doing something that takes 9 seconds...
Timeout!


### 2. threading

By defining a ```timelimit``` decorator, we create a threading subclass that runs a function and raises a ```TimeoutException``` exception if it excedes the time limit specified in ```timeout```.

<div class="alert alert-danger">
  <strong>Note!</strong> This doesn't actually stop the function after the timeout. It leaves it running in a separate thread, still consuming resources.  This is NOT a recommended approach.
</div>

In [3]:
import sys


class TimeoutException(Exception):
    pass

def timelimit(timeout):
    """
    Run function with the given timeout. If function doesn't finish within the timeout, raise TimeoutException
    """
    def internal(function):
        def internal2(*args, **kw):
            import threading
            class FuncThread(threading.Thread):
                def __init__(self):
                    threading.Thread.__init__(self)
                    self.result = None
                    self.error = None
                
                def run(self):
                    try:
                        self.result = function(*args, **kw)
                    except:
                        self.error = sys.exc_info()[0]
            c = FuncThread()
            c.start()
            c.join(timeout)
            if c.isAlive():
                #c._stop()  # mark the thread as stopped; raises AssertionError in newer versions of Python
                raise TimeoutException
            if c.error:
                raise c.error
            return c.result
        return internal2
    return internal

In [4]:
import time
from random import randint
from retrying import retry


@retry(stop_max_attempt_number=6)  # stop after 6 attempts
@timelimit(1)  
def long_running_function():
        sleep_time = randint(0,9)
        print("Doing something that takes " + str(sleep_time) + " seconds...")
        time.sleep(sleep_time)
        print("Finished!")

        
try:
    long_running_function()
except TimeoutException:
    print("Timeout!")

Doing something that takes 3 seconds...
Doing something that takes 8 seconds...
Doing something that takes 9 seconds...
Finished!
Doing something that takes 3 seconds...
Doing something that takes 8 seconds...
Doing something that takes 2 seconds...
Finished!
Timeout!
Finished!
Finished!
Finished!
Finished!


### 3. multiprocessing.Process

This uses ```multiprocessing.Process``` to run a wrapped function in a separate process, store the result in a queue, and terminate the process if it reaches the timeout.

<div class="alert alert-warning">
  <strong>Note!</strong> Process creation is based on the fork system call which creates a copy of the whole environment.  This uses a lot of memory and takes a considerable amount of time.  For smaller applications, this is OK.  But for large applications, this is NOT a recommended approach.
</div>

In [5]:
from functools import wraps
from multiprocessing import Process, Queue
 
class TimeoutException(Exception):
    pass
 
def timelimit(timeout):
    def wrap_function(func):
        @wraps(func)
        def __wrapper(*args, **kwargs):
            def queue_wrapper(args, kwargs):
                q.put(func(*args, **kwargs))
 
            q = Queue()
            p = Process(target=queue_wrapper, args=(args, kwargs))
            p.start()
            p.join(timeout)
            if p.is_alive():
                p.terminate()
                p.join()
                raise TimeoutException()
            p.terminate()
            return q.get()
        return __wrapper
    return wrap_function

In [6]:
import time
from random import randint
from retrying import retry


@retry(stop_max_attempt_number=6)  # stop after 6 attempts
@timelimit(1)  
def long_running_function():
        sleep_time = randint(0,9)
        print("Doing something that takes " + str(sleep_time) + " seconds...")
        time.sleep(sleep_time)
        print("Finished!")

        
try:
    long_running_function()
except TimeoutException:
    print("Timeout!")

Doing something that takes 7 seconds...
Doing something that takes 3 seconds...
Doing something that takes 4 seconds...
Doing something that takes 1 seconds...
Doing something that takes 6 seconds...
Doing something that takes 3 seconds...
Timeout!


### 4. multiprocessing.Pool

```multiprocessing.Pool``` manages the number of processes as well as job collection.

<div class="alert alert-warning">
  <strong>Note!</strong> This is the same approach as above.  We don't gain anything except the ability to create multiple processes.
</div>

In [7]:
import time
from random import randint
from retrying import retry


def long_running_function():
    sleep_time = randint(0,9)
    print("Doing something that takes " + str(sleep_time) + " seconds...")
    time.sleep(sleep_time)
    print("Finished!")


import multiprocessing

@retry(stop_max_attempt_number=6)  # stop after 6 attempts
def function():
    pool = multiprocessing.Pool(1, maxtasksperchild=1)
    result = pool.apply_async(long_running_function)
    pool.close()

    try:
        s = result.get(1)
        print(s)
    except multiprocessing.TimeoutError:
        pool.terminate()
        print("Timeout!")
        raise TimeoutError

try:
    function()
except TimeoutError:
    print("Timeout!")

Doing something that takes 8 seconds...
Timeout!
Doing something that takes 1 seconds...
Timeout!
Doing something that takes 3 seconds...
Timeout!
Doing something that takes 9 seconds...
Timeout!
Doing something that takes 3 seconds...
Timeout!
Doing something that takes 1 seconds...
Timeout!
Timeout!


### 5. Pebble library

The ```Pebble``` library is a wrapper of the built-in ```multiprocessing``` library and is the best implementation for handling timeouts in functions.

<div class="alert alert-success">
  <strong>Note!</strong> While this requires a third-party library, this is the recommended approach.
</div>

In [10]:
import time
from random import randint
from retrying import retry
from pebble import concurrent


class TimeoutException(Exception):
    pass

@concurrent.process(timeout=1)
def long_running_function():
        sleep_time = randint(0,9)
        print("Doing something that takes " + str(sleep_time) + " seconds...")
        time.sleep(sleep_time)
        print("Finished!")


@retry(stop_max_attempt_number=6)  # stop after 6 attempts
def function():
    result = long_running_function().result()  # blocks until results are ready


try:
    function()
except Exception as error:
    print("Timeout! %s" % error)

Doing something that takes 3 seconds...
Doing something that takes 5 seconds...
Doing something that takes 2 seconds...
Doing something that takes 3 seconds...
Doing something that takes 4 seconds...
Doing something that takes 9 seconds...
Timeout! ('Task Timeout', 1)
